In [58]:
# import dependencies
from pprint import pprint
import pandas as pd
import json
import numpy as np

In [69]:
# load amzn data and format
amzn_raw_df = pd.read_csv('Resources/amzn_stock_data_raw.csv')
amzn_df = amzn_raw_df[['t','n','v','vw','o','c','h','l']].rename(columns={'v':'volume',\
                                                            'vw':'volume weighted average price',\
                                                            'o':'open', 'c':'close',\
                                                            'h':'high', 'l':'low',\
                                                            't':'Date','n':'transaction count'})
amzn_df.sort_values('Date', ascending=True, inplace=True)
amzn_df['Date'] = pd.to_datetime(amzn_df['Date'], origin='unix', unit='ms').dt.date

# displaying amzn_df
amzn_df.head()

# checking date type
# date_type = type(amzn_df['Date'][0])
# print(f'date column value are in format: {date_type}.')

,Date,transaction count,volume,volume weighted average price,open,close,high,low
0,2020-01-02,102920,4035910.0,1882.8283,1875.00,1898.01,1898.0100,1864.1500
1,2020-01-03,99633,3766604.0,1875.4284,1864.50,1874.97,1886.1965,1864.5000
2,2020-01-06,104160,4065698.0,1891.9954,1860.00,1902.88,1903.6900,1860.0000
3,2020-01-07,98636,4134010.0,1903.2183,1904.50,1906.86,1913.8900,1892.0433
4,2020-01-08,95706,3511966.0,1900.4983,1898.04,1891.97,1910.9999,1886.4448


In [80]:
# load covid data
us_covid_raw_df = pd.read_csv('../Covid/US_covid.csv')

# checking date format
# print(type(us_covid_raw_df.loc[0,'Date']))
# print(us_covid_raw_df.dtypes)

# changing date format
# us_covid_raw_df['Date'] = pd.to_datetime(us_covid_raw_df['Date'])
# us_covid_raw_df['Date'] = us_covid_raw_df['Date'].dt.strftime('%Y-%m-%d')
us_covid_raw_df['Date'] = pd.to_datetime(us_covid_raw_df['Date']).dt.date

# checking date type again
# print(type(us_covid_raw_df.loc[0,'Date']))
# print(us_covid_raw_df.dtypes)
# print(amzn_df.dtypes)

us_covid_raw_df.sort_values('Date', ascending=True)

us_covid_raw_df.head()
# us_covid_raw_df.tail()


,Date,Country,New Cases,Cumulative Cases,New Deaths,Cumulative Deaths
0,2020-01-03,United States of America,0,0,0,0
1,2020-01-04,United States of America,0,0,0,0
2,2020-01-05,United States of America,0,0,0,0
3,2020-01-06,United States of America,0,0,0,0
4,2020-01-07,United States of America,0,0,0,0


In [81]:
# us_covid_raw_df.to_csv('test/covid.csv')
# amzn_df.to_csv('test/amzn.csv')

In [82]:
# drop date rows where there are no new reported cases (non-reporting days)
us_covid_df = us_covid_raw_df.loc[us_covid_raw_df['New Cases'] != 0]

us_covid_df.head()

,Date,Country,New Cases,Cumulative Cases,New Deaths,Cumulative Deaths
17,2020-01-20,United States of America,1,1,0,0
22,2020-01-25,United States of America,8,9,0,0
28,2020-01-31,United States of America,7,16,0,0
35,2020-02-07,United States of America,10,26,0,0
42,2020-02-14,United States of America,10,36,0,0


In [83]:
# covidraw index 0 is 2022-01-03
print(us_covid_raw_df.iloc[0,0])
print(type(us_covid_raw_df.iloc[0,0]))

# amzn index 1 is 2022-01-03
print(amzn_df.iloc[1,0])
print(type(amzn_df.iloc[1,0]))


2020-01-03
<class 'datetime.date'>
2020-01-03
<class 'datetime.date'>


In [88]:
# merging data on date
all_dates_df = pd.merge(us_covid_raw_df, amzn_df, how='inner', on='Date')
# all_dates_df.to_csv('Resources/all_dates_combined.csv')

# getting only dates with new reported cases/deaths
filtered_df = all_dates_df.loc[all_dates_df["New Cases"] != 0]
filtered_df.head()

,Date,Country,New Cases,Cumulative Cases,New Deaths,Cumulative Deaths,transaction count,volume,volume weighted average price,open,close,high,low
19,2020-01-31,United States of America,7,16,0,0,378587,15565783.0,2032.9560,2051.47,2008.72,2055.7200,2002.270
24,2020-02-07,United States of America,10,26,0,0,135486,5095347.0,2077.4163,2041.99,2079.28,2098.5300,2038.100
29,2020-02-14,United States of America,10,36,0,0,74726,2606169.0,2139.4129,2155.68,2134.87,2159.0352,2125.890
33,2020-02-21,United States of America,16,52,0,0,131586,4637440.0,2107.7886,2142.15,2095.97,2144.5500,2088.000
38,2020-02-28,United States of America,31,83,0,0,310386,9493797.0,1850.4402,1814.63,1883.75,1889.7600,1811.129
